<a href="https://colab.research.google.com/github/LeoVogiatzis/GNN_based_NILM/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git


In [ ]:
!git clone https://github.com/shenweichen/GraphEmbedding.git


In [ ]:
import networkx as nx
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
import numpy as np
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
import sys
print(sys.path)
! pip install Node2Vec

In [ ]:
G = nx.read_graphml('dishwaser_20.graphml')

In [ ]:
from node2vec import Node2Vec
# load graph from networkx library
G = nx.read_graphml('dishwaser_20.graphml')

# convert nodes from int to str format
keys = np.arange(0,len(G.nodes()))
values = [str(i) for i in keys]
dic = dict(zip(keys, values))
H = nx.relabel_nodes(G, dic)

# train the model and generate embeddings

model = Node2Vec(H, walk_length=10, num_walks=80, workers=2)
embeddings = model.fit(window=5,iter=3)

# embeddings = model.get_embeddings()

Computing transition probabilities:   0%|          | 0/410 [00:00<?, ?it/s]

In [ ]:
for node, _ in model.most_similar('100'):
    # Show only players
    if len(node) > 3:
        print(node)

In [ ]:
import networkx as nx
import numpy as np
import torch
from node2vec import Node2Vec
  
# retrieve the labels for each node
labels = np.asarray([G.nodes[i]['state'] for i in G.nodes]).astype(np.int64)

# create edge index from 
adj = nx.to_scipy_sparse_matrix(G).tocoo()
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row, col], dim=0)

# using degree as embedding
embeddings = np.array(list(dict(G.degree()).values()))

# normalizing degree values
scale = StandardScaler()
embeddings = scale.fit_transform(embeddings.reshape(-1,1))

array([   2,   26,   33,   52,   55,   59,   64,   68,   81,   90,   92,
         95,  219,  227,  230,  241,  244,  247,  249,  253,  265,  281,
        287,  291,  296,  302,  305,  311,  316,  321,  325,  331,  345,
        358,  363,  365,  368,  373,  375,  380,  382,  387,  389,  397,
        401,  411,  413,  415,  418,  422,  424,  426,  432,  438,  444,
        455,  461,  497,  545,  577,  613,  621,  623,  631,  635,  639,
        643,  686,  738,  757,  777,  792,  798,  802,  807,  810,  820,
        822,  825,  831,  834,  837,  839,  842,  845,  849,  851,  855,
        858,  863,  872,  876,  878,  885,  890,  902,  908,  911,  915,
        917,  921,  923,  926,  928,  930,  936,  952,  954,  962,  980,
        989,  992, 1012, 1015, 1022, 1025, 1048, 1057, 1068, 1124, 1127,
       1129, 1322, 1354, 1404, 1468, 1478, 1486, 1490, 1502, 1511, 1562,
       1569, 1573, 1711, 2227, 2347, 2368, 2424, 2441, 2839, 2880, 2894,
       2923, 3044, 3436, 3468, 3627, 3854, 3874, 38

In [ ]:

import torch
import pandas as pd
from torch_geometric.data import InMemoryDataset, Data
from sklearn.model_selection import train_test_split
import torch_geometric.transforms as T

# custom dataset
class NilmDataset(InMemoryDataset):
    def __init__(self, transform=None):
        super(NilmDataset, self).__init__('.', transform, None, None)

        data = Data(edge_index=edge_index)
        
        data.num_nodes = G.number_of_nodes()
        
        # embedding 
        data.x = torch.from_numpy(embeddings).type(torch.float32)
        
        # labels
        y = torch.from_numpy(labels).type(torch.long)
        data.y = y.clone().detach()
        
        data.num_classes = 2

        # splitting the data into train, validation and test
        X_train, X_test, y_train, y_test = train_test_split(pd.Series(np.asarray([int(i) for i in G.nodes()]).astype(np.int64)), 
                                                            pd.Series(labels),
                                                            test_size=0.30, 
                                                            random_state=42)
        
        n_nodes = G.number_of_nodes()
        
        # create train and test masks for data
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[X_train.index] = True
        test_mask[X_test.index] = True
        data['train_mask'] = train_mask
        data['test_mask'] = test_mask

        self.data, self.slices = self.collate([data])

    def _download(self):
        return

    def _process(self):
        return

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)
    
dataset = NilmDataset()
data = dataset[0]

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# GCN model with 2 layers 
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(data.num_features, 16)
        self.conv2 = GCNConv(16, int(data.num_classes))

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data =  data.to(device)

model = Net().to(device) 

In [ ]:
torch.manual_seed(42)

optimizer_name = "Adam"
lr = 1e-1
optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
epochs = 200

def train():
  model.train()
  optimizer.zero_grad()
  F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
  optimizer.step()

@torch.no_grad()
def test():
  model.eval()
  logits = model()
  mask1 = data['train_mask']
  pred1 = logits[mask1].max(1)[1]
  acc1 = pred1.eq(data.y[mask1]).sum().item() / mask1.sum().item()
  mask = data['test_mask']
  pred = logits[mask].max(1)[1]
  acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
  return acc1,acc

for epoch in range(1, epochs):
  train()

train_acc,test_acc = test()

print('#' * 70)
print('Train Accuracy: %s' %train_acc )
print('Test Accuracy: %s' % test_acc)
print('#' * 70)

######################################################################
Train Accuracy: 0.5261324041811847
Test Accuracy: 0.43902439024390244
######################################################################
